In [ ]:
from dask.distributed import Client
client = Client(n_workers=4, threads_per_worker=4, memory_limit='4GB')
# link to dashboard
print(client.dashboard_link)

import xarray as xr
import zarr
import os
import glob

In [ ]:
netcdf_dir = r"path/to/netcdf"
zarr_dir = r"/path/to/zarr.zarr"

In [ ]:
filelist = glob.glob(os.path.join(netcdf_dir,"*.nc"))
ds = xr.open_mfdataset(filelist[1:2], combine='by_coords',
                       chunks={'time': 1, 'latitude' : 1080, 'longitude': 1080},
                       parallel=True)

In [ ]:
encoding = {vname: {
    'compressor': zarr.Blosc(cname='zstd', clevel=5),
    } for vname in ds.data_vars}
ds.attrs['reprocessing'] = "rechunked and compressed with zarr using xarray 02/2024"

ds.to_zarr(zarr_dir, encoding=encoding, consolidated=True, mode='w', compute=True)

In [ ]:
client.close()